1. Read the csv file
2. Drop Date feild
3. If quality is 'Good' mark it as 1, if its 'Bad' mark it as 2 
4. Make  df of the Good and Bad parts separately and drop the Quality column
5. Obtain the values of the good parts and prepare training and testing set.

In [380]:
import pandas as pd
df = pd.read_csv('anomaly.csv')
print(df.head())

       Date  Power  Detector Quality
0  1-Jan-16     96         8    Good
1  2-Jan-16     96        10    Good
2  3-Jan-16     91         8    Good
3  4-Jan-16     97         9    Good
4  5-Jan-16     91        11    Good


In [381]:
df.drop(['Date'], axis = 1, inplace= True)
df.dropna(inplace=True, axis=1)

In [382]:
df.Quality[df.Quality=='Good'] = 1
df.Quality[df.Quality=='Bad'] = 2


C:\Users\vijay\AppData\Local\Temp\ipykernel_23204\482985629.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Quality[df.Quality=='Good'] = 1
C:\Users\vijay\AppData\Local\Temp\ipykernel_23204\482985629.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Quality[df.Quality=='Bad'] = 2


In [383]:
good_mask = df['Quality']== 1 #All good to be True for good data points
bad_mask = df['Quality']== 2 #All values False for bad data points

In [384]:
df.drop(['Quality'], axis=1, inplace=True)
print(df)

      Power  Detector
0        96         8
1        96        10
2        91         8
3        97         9
4        91        11
...     ...       ...
1456     93         8
1457     91        10
1458     95         8
1459     91        10
1460    100        12

[1461 rows x 2 columns]


In [385]:
df_good = df[good_mask]
df_bad = df[bad_mask]
print(df_good)

      Power  Detector
0        96         8
1        96        10
2        91         8
3        97         9
4        91        11
...     ...       ...
1456     93         8
1457     91        10
1458     95         8
1459     91        10
1460    100        12

[1054 rows x 2 columns]


In [386]:
print(f"Good count: {len(df_good)}")
print(f"Bad count: {len(df_bad)}")

Good count: 1054
Bad count: 407


In [387]:
x_good = df_good.values
x_bad = df_bad.values
print(x_good)

[[ 96   8]
 [ 96  10]
 [ 91   8]
 ...
 [ 95   8]
 [ 91  10]
 [100  12]]


In [388]:
total_value = pd.concat([df_good,df_bad], axis=0, ignore_index=True)
total1 = total_value.values


In [389]:
from sklearn.model_selection import train_test_split
# split 75 percent of data into training set and other 25 percent to testing set and set the randomizing during split as 42
x_good_train, x_good_test = train_test_split(
        x_good, test_size=0.25, random_state=42)

In [390]:
print(f"Good train count: {len(x_good_train)}")
print(f"Good test count: {len(x_good_test)}")
print(f"Good test count: {len(x_good)}")

Good train count: 790
Good test count: 264
Good test count: 1054


In [391]:
########################################################################################################

In [392]:
from sklearn import metrics
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [393]:
print(x_good.shape)

(1054, 2)


In [394]:
model = Sequential()
model.add(Dense(10, input_dim=x_good.shape[1], activation = 'relu'))
model.add(Dense(3,activation = 'relu'))
model.add(Dense(10,activation = 'relu'))
model.add(Dense(x_good.shape[1]))
model.compile(loss='mean_squared_error', optimizer = 'adam')
model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_44 (Dense)            (None, 10)                30        
                                                                 
 dense_45 (Dense)            (None, 3)                 33        
                                                                 
 dense_46 (Dense)            (None, 10)                40        
                                                                 
 dense_47 (Dense)            (None, 2)                 22        
                                                                 
Total params: 125 (500.00 Byte)
Trainable params: 125 (500.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [395]:
model.fit(x_good_train,x_good_train, verbose=1, epochs=100)

Epoch 1/100


25/25 [==============================] - 1s 3ms/step - loss: 4862.3315
Epoch 2/100
25/25 [==============================] - 0s 3ms/step - loss: 4661.8232
Epoch 3/100
25/25 [==============================] - 0s 2ms/step - loss: 4571.7671
Epoch 4/100
25/25 [==============================] - 0s 2ms/step - loss: 4535.1631
Epoch 5/100
25/25 [==============================] - 0s 2ms/step - loss: 4484.4717
Epoch 6/100
25/25 [==============================] - 0s 2ms/step - loss: 4341.0815
Epoch 7/100
25/25 [==============================] - 0s 2ms/step - loss: 4012.5332
Epoch 8/100
25/25 [==============================] - 0s 2ms/step - loss: 3421.5869
Epoch 9/100
25/25 [==============================] - 0s 2ms/step - loss: 2580.9429
Epoch 10/100
25/25 [==============================] - 0s 2ms/step - loss: 1628.4282
Epoch 11/100
25/25 [==============================] - 0s 2ms/step - loss: 779.6790
Epoch 12/100
25/25 [==============================] - 0s 2ms/step - loss: 234.0431
Epoch 13/100
25

In [396]:
pred = model.predict(x_good_test)
# the test data is sent to model and prediction is done
score1 = np.sqrt(metrics.mean_squared_error(pred,x_good_test))
# then the mean squared error of the test set and the model predicted result of test set is stored in score1

9/9 [==============================] - 0s 2ms/step


In [397]:
pred = model.predict(x_good_train)
score2 = np.sqrt(metrics.mean_squared_error(pred,x_good_train))
# x_good is the entire data set of the good entries
pred = model.predict(x_good)
score3 = np.sqrt(metrics.mean_squared_error(pred,x_good))

33/33 [==============================] - 0s 2ms/step


In [398]:
pred = model.predict(x_bad)
score4 = np.sqrt(metrics.mean_squared_error(pred,x_bad))

13/13 [==============================] - 0s 2ms/step


In [399]:
print(f"In test set of Good Score (RMSE): {score1}".format(score1))
print(f"In train set Good Score (RMSE): {score2}")
print(f"Entire set Good Score (RMSE): {score3}")
print(f"Bad entire set Score (RMSE): {score4}")

In test set of Good Score (RMSE): 1.0971952017794502
In train set Good Score (RMSE): 1.0952625030832648
Entire set Good Score (RMSE): 1.0957468993752995
Bad entire set Score (RMSE): 2.7212279102600623


In [400]:
pred = model.predict(total1)
score5 = np.sqrt(metrics.mean_squared_error(pred,total1))
print(score5)

46/46 [==============================] - 0s 2ms/step
1.7114513696268348
